In [27]:
api_key = "RGAPI-11c8885f-7b92-4e0c-b084-b457e7398203"

In [3]:
import requests


In [163]:
#test de converteur de nombre romain en int 
class roman_to_int(object):
   def romanToInt(self, s):
      """
      :type s: str
      :rtype: int
      """
      roman = {'I':1,'V':5,'X':10,'L':50,'C':100,'D':500,'M':1000,'IV':4,'IX':9,'XL':40,'XC':90,'CD':400,'CM':900}
      i = 0
      num = 0
      while i < len(s):
         if i+1<len(s) and s[i:i+2] in roman:
            num+=roman[s[i:i+2]]
            i+=2
         else:
            #print(i)
            num+=roman[s[i]]
            i+=1
      return num


In [164]:
## récupère puis stocke le PUUID du summoner
## toujours besoin du convertisseur de chiffre romain

def ranking_information (summoner_list_name:list):
    global list_summoner_rank
    list_summoner_rank = []

    for summoner in summoner_list_name:
        data_summoner = requests.get('https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+ summoner +'?api_key='+api_key).json()
        if data_summoner is not None and "id" in data_summoner:
            id_summoner = data_summoner.get("id")
            puuid_summoner = data_summoner.get("puuid")
            #print(puuid_summoner,id_summoner)
  
            ## récupère les matchs (les derniers) du summoner (ici 5 matchs)
            ## ressource utile les ID des queues : https://static.developer.riotgames.com/docs/lol/queues.json
            ## récupère la division, le tier et les LP du compte
            
            data_rank = requests.get('https://euw1.api.riotgames.com/lol/league/v4/entries/by-summoner/'+ id_summoner +'?api_key='+api_key).json()
            
            presence_of_soloq_data = False
            for data_SOLOQ in data_rank:
                #if 'RANKED_SOLO_5x5' in data_SOLOQ['queueType']:
                if data_SOLOQ['queueType'] == "RANKED_SOLO_5x5":
                    print(summoner,data_SOLOQ['tier'],data_SOLOQ['rank'],data_SOLOQ['leaguePoints'], "LP")
                    rank = roman_to_int().romanToInt(data_SOLOQ['rank'])  
                    list_summoner_rank.append((summoner,data_SOLOQ['tier'],rank,data_SOLOQ['leaguePoints']))
                    presence_of_soloq_data = True
            if not presence_of_soloq_data:
                print(f"No data / No SOLO Q for this account : {summoner}")
                list_summoner_rank.append((summoner,"none","0", "0"))
        else:
            print(f"Failed to retrieve data for summoner {summoner}")
            
summoner_list = ["SaintLoutr","Draeknof","Haukeas Grémory","Caio73","charlos92604","7689000","Alrsay","TheK1ngPotato","Blarckful"]
#summoner_list = ["SaintLoutr","TheK1ngPotato","Blarckful"]

ranking_information(summoner_list)
print(list_summoner_rank)



SaintLoutr SILVER III 54 LP
Draeknof GOLD III 25 LP
Haukeas Grémory GOLD III 15 LP
Caio73 BRONZE I 33 LP
charlos92604 BRONZE II 52 LP
7689000 BRONZE IV 0 LP
Alrsay GOLD II 0 LP
No data / No SOLO Q for this account : TheK1ngPotato
Blarckful SILVER II 56 LP
[('SaintLoutr', 'SILVER', 3, 54), ('Draeknof', 'GOLD', 3, 25), ('Haukeas Grémory', 'GOLD', 3, 15), ('Caio73', 'BRONZE', 1, 33), ('charlos92604', 'BRONZE', 2, 52), ('7689000', 'BRONZE', 4, 0), ('Alrsay', 'GOLD', 2, 0), ('TheK1ngPotato', 'none', '0', '0'), ('Blarckful', 'SILVER', 2, 56)]


In [143]:
##vérifie le résultat des game avec l'ID du match et le PUUID du summoner 
def result_match (summoner_list):
    
    for summoner in summoner_list:
        data_summoner = requests.get('https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+ summoner +'?api_key='+api_key).json()
        puuid_summoner = data_summoner.get("puuid")
        matches_id = requests.get('https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/'+ puuid_summoner +'/ids?queue='+'420'+'&start=0&count=1&api_key='+api_key).json()
        print(summoner)

        
        for match_id in matches_id:
            match_info = requests.get("https://europe.api.riotgames.com/lol/match/v5/matches/"+ match_id +"?api_key=" + api_key).json()
            for player in (match_info['info']['participants']):
                if player['puuid'] == puuid_summoner:
                    if player['win'] == True:
                        print("WIN")
                    else:
                        print("LOOSE")
                
    list_player = []

summoner_list = ["SaintLoutr","Draeknof","Haukeas Grémory"]
result_match(summoner_list)

SaintLoutr
LOOSE
Draeknof
LOOSE
Haukeas Grémory
LOOSE


In [166]:
#ajout des données de rank des compte dans une base de données

import sqlite3

list_summoner_rank = [('SaintLoutr', 'SILVER', 3, 54), ('Draeknof', 'GOLD', 3, 25), ('Haukeas Grémory', 'GOLD', 3, 15), ('Caio73', 'BRONZE', 1, 33), ('charlos92604', 'BRONZE', 2, 52), ('7689000', 'BRONZE', 4, 0), ('Alrsay', 'GOLD', 2, 0), ('TheK1ngPotato', 'none', '0', '0'), ('Blarckful', 'SILVER', 2, 56)]

def insert_into_leaderboard(SummonerName, Tier, Rank, LP):
  try:
    conn = sqlite3.connect('leaderboard.db')
    cur = conn.cursor()
    print("Connexion réussie à SQLite")
    sql = "INSERT INTO leaderboard (SummonerName, Tier, Rank, LP) VALUES (?, ?, ?, ?)"
    value = (SummonerName, Tier, Rank, LP)
    cur.execute(sql, value)
    conn.commit()
    print("Enregistrement inséré avec succès dans la table leaderboard")
    cur.close()
    conn.close()
    print("Connexion SQLite est fermée")
  except sqlite3.Error as error:
    print("Erreur lors de l'insertion dans la table leaderboard", error)

for summoner in list_summoner_rank:
    SummonerName, Tier, Rank, LP = summoner
    insert_into_leaderboard(SummonerName, Tier, Rank, LP)

Connexion réussie à SQLite
Enregistrement inséré avec succès dans la table leaderboard
Connexion SQLite est fermée
Connexion réussie à SQLite
Enregistrement inséré avec succès dans la table leaderboard
Connexion SQLite est fermée
Connexion réussie à SQLite
Enregistrement inséré avec succès dans la table leaderboard
Connexion SQLite est fermée
Connexion réussie à SQLite
Enregistrement inséré avec succès dans la table leaderboard
Connexion SQLite est fermée
Connexion réussie à SQLite
Enregistrement inséré avec succès dans la table leaderboard
Connexion SQLite est fermée
Connexion réussie à SQLite
Enregistrement inséré avec succès dans la table leaderboard
Connexion SQLite est fermée
Connexion réussie à SQLite
Enregistrement inséré avec succès dans la table leaderboard
Connexion SQLite est fermée
Connexion réussie à SQLite
Enregistrement inséré avec succès dans la table leaderboard
Connexion SQLite est fermée
Connexion réussie à SQLite
Enregistrement inséré avec succès dans la table leade

In [ ]:
#suppresion de la table

try:
   conn = sqlite3.connect('leaderboard.db')
   cur = conn.cursor()
   print("Connexion réussie à SQLite")
   cur.execute("delete from leaderboard")
   conn.commit()
   print("leaderboard supprimé")
   cur.close()
   conn.close()
   print("Connexion SQLite est fermée")
except sqlite3.Error as error:
   print("Erreur ", error)

In [170]:
#sorting de la liste

try:
   conn = sqlite3.connect('leaderboard.db')
   cur = conn.cursor()
   print("Connexion réussie à SQLite")
   cur.execute("SELECT *FROM leaderboard ORDER BY CASE WHEN tier = 'CHALLENGER' THEN 1 WHEN tier = 'GRANDMASTER' THEN 2 WHEN tier = 'MASTER' THEN 3 WHEN tier = 'DIAMOND' THEN 4 WHEN tier = 'PLATINIUM' THEN 5 WHEN tier = 'GOLD' THEN 6 WHEN tier = 'SILVER' THEN 7 WHEN tier = 'BRONZE' THEN 8 WHEN tier = 'IRON' THEN 9 ELSE 10 END,Rank asc,lp desc;")
   conn.commit()
   print("leaderboard sorted")
   cur.close()
   conn.close()
   print("Connexion SQLite est fermée")
except sqlite3.Error as error:
   print("Erreur ", error)

Connexion réussie à SQLite
leaderboard sorted
Connexion SQLite est fermée
